In [3]:
import random
import json
import os

import torch

import train
from model import NeuralNetwork
from nltk_utils import bag_of_words, tokenize

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

global all_words, tags, model

In [8]:
def load_intents():
    with open('intents.json', 'r') as json_data:
        intents = json.load(json_data)
    return intents

In [9]:
def init():
    global all_words, tags, model

    if not os.path.exists('data.pth'):
        train.train_data()
        
    FILE = "data.pth"
    data = torch.load(FILE)

    input_size = data["input_size"]
    hidden_size = data["hidden_size"]
    output_size = data["output_size"]
    all_words = data['all_words']
    tags = data['tags']
    model_state = data["model_state"]

    model = NeuralNetwork(input_size, hidden_size, output_size).to(device)
    model.load_state_dict(model_state)
    model.eval()

In [20]:
def main():
    init()
    intents = load_intents()

    bot_name = "Sam"
    print("Let's chat! (type 'quit' to exit)")
    while True:
        # sentence = "do you use credit cards?"
        sentence = input("You: ")
        if sentence == "quit":
            break

        sentence = tokenize(sentence)
        X = bag_of_words(sentence, all_words)
        X = X.reshape(1, X.shape[0])
        X = torch.from_numpy(X).to(device)

        output = model(X)
        _, predicted = torch.max(output, dim=1)

        tag = tags[predicted.item()]

        probs = torch.softmax(output, dim=1)
        prob = probs[0][predicted.item()]
        if prob.item() > 0.8:
            for intent in intents['intents']:
                if tag == intent["tag"]:
                    print(f"{bot_name}: {random.choice(intent['responses'])}")
        else:
            print(f"{bot_name}: I do not understand...")

In [21]:
main()

Let's chat! (type 'quit' to exit)
Sam: Hi there, how can I help?
Sam: Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
Sam: Our university offers Information Technology, computer Engineering, Mechanical engineering,Chemical engineering, Civil engineering and extc Engineering.
Sam: XYZ is college principal and if you need any help then call your branch hod first.That is more appropriate
Sam: I do not understand...
